In [10]:
import numpy as np
import pandas as pd
import random
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [11]:
# datapreprocess
Greek = pd.read_csv(r'Greek.csv')
inforce = pd.read_csv('inforce.csv')
df = pd.merge(Greek, inforce)
age = (df['currentDate']- df['birthDate'])/365
ttm = (df['matDate']- df['currentDate'])/365
df = df[['recordID','gender','gmwbBalance','productType','gbAmt','FundValue1', 'FundValue2', 'FundValue3', 'FundValue4', 'FundValue5',
       'FundValue6', 'FundValue7', 'FundValue8', 'FundValue9', 'FundValue10', 'fmv']]
df["age"] = age
df['ttm'] = ttm
df = df[['recordID','gender','gmwbBalance','productType','gbAmt','FundValue1', 'FundValue2', 'FundValue3', 'FundValue4', 'FundValue5',
       'FundValue6', 'FundValue7', 'FundValue8', 'FundValue9', 'FundValue10', 'fmv']]
dummy = pd.get_dummies(df[['recordID','productType','gender']], drop_first=True)

df = pd.merge(df, dummy)
df = df.drop(['productType','gender','recordID'], axis=1)

colname = ['gmwbBalance', 'gbAmt', 'FundValue1', 'FundValue2', 'FundValue3',
       'FundValue4', 'FundValue5', 'FundValue6', 'FundValue7', 'FundValue8',
       'FundValue9', 'FundValue10',  'productType_ABRU',
       'productType_ABSU', 'productType_DBAB', 'productType_DBIB',
       'productType_DBMB', 'productType_DBRP', 'productType_DBRU',
       'productType_DBSU', 'productType_DBWB', 'productType_IBRP',
       'productType_IBRU', 'productType_IBSU', 'productType_MBRP',
       'productType_MBRU', 'productType_MBSU', 'productType_WBRP',
       'productType_WBRU', 'productType_WBSU', 'gender_M']

norm = preprocessing.normalize(df[colname])
df[colname] = norm
df['fmv'] = df['fmv']/1000
df.to_csv('data.csv',index = False)

In [1]:
%%file script2.py
#!/usr/bin/env python
from mpi4py import MPI
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()


if rank == 0:
    data = pd.read_csv("data.csv")
    split_rate = 0.9
    Train = data.iloc[:int(data.shape[0]*split_rate),:]
    Test = data.iloc[int(data.shape[0]*split_rate):,:]
    
else:
    # data = None
    Train = None
    Test = None
# data = comm.bcast(data,root = 0)
Train = comm.bcast(Train,root = 0)
Test = comm.bcast(Test,root = 0)

y_test = Test['fmv']

X_test = Test.drop(['fmv'],axis = 1)
print(rank)
y_test_pre_list = []
error_model_list = []
for i in range(3):
    Train_sample = Train.sample(frac = 1, replace = True)
    y_train_sample = Train_sample['fmv']
    X_train_sample = Train_sample.drop(['fmv'],axis = 1)
    
    model = xgb.XGBRegressor()
    model.fit(X_train_sample,y_train_sample)
    
    y_train_sample_pre = model.predict(X_train_sample)
    error_model = y_train_sample-y_train_sample_pre
    error_model_list.append(error_model)
    y_test_pre = model.predict(X_test)
    y_test_pre_list.append(y_test_pre)
    print('The rank',rank,'iteration',i)
    
error_l = comm.gather(error_model_list, root=0)
y_test_pre_l = comm.gather(y_test_pre_list, root=0)
if rank == 0:
    re = np.array(y_test_pre_l)
    y_test_pre_list_ = np.transpose(re.reshape(re.shape[0]*re.shape[1],re.shape[2]))
    re2 = np.array(error_l)
    error_model_list_ = np.transpose(re2.reshape(re2.shape[0]*re2.shape[1],re2.shape[2]))
    # print(error_model_list_.shape)
    # print(y_test_pre_list_.shape)
    error_random = np.random.choice(np.array(error_model_list_).flatten(),(y_test_pre_list_.shape))
    y_test_revise_list   = y_test_pre_list_ +   error_random
    
    Left_list = np.quantile(np.transpose(y_test_revise_list),0.025,axis = 0)
    # print(Left_list.shape)
    Right_list = np.quantile(np.transpose(y_test_revise_list),0.975,axis = 0)
    # print(Right_list.shape)
    Bool_list = (y_test >= Left_list) & (y_test< Right_list)
    
    diff = Right_list-Left_list
    results_10_1000 = pd.DataFrame([])
    results_10_1000['Bool_list'] = Bool_list
    results_10_1000['width'] = diff
    results_10_1000['test'] = Test['fmv'].copy()
    pd.DataFrame(results_10_1000).to_csv('./result/results_10_1000.csv',index = False)
    print(f"accuracy: {Bool_list.mean():.2f}")
 

Writing script2.py


In [11]:
import numpy as np
import pandas as pd
import xgboost as xgb

data = pd.read_csv("data.csv")
split_rate = 0.8
Train = data.iloc[:int(data.shape[0]*split_rate),:]
Test = data.iloc[int(data.shape[0]*split_rate):,:]

y_test = Test['fmv']
X_test = Test.drop(['fmv'],axis = 1)

Train_sample = Train.sample(frac = 1, replace = True)
y_train_sample = Train_sample['fmv']
X_train_sample = Train_sample.drop(['fmv'],axis = 1)

model = xgb.XGBRegressor()
model.fit(X_train_sample,y_train_sample)

y_train_sample_pre = model.predict(X_train_sample)
error_model = y_train_sample-y_train_sample_pre

y_test_pre = model.predict(X_test)
print("error:",*["%.8f"%(x) for x in error_model])
print("y_test_pre:",*["%.8f"%(x) for x in y_test_error])



error: -26.05419810 -4.97330495 2.86480886 -4.93463389 14.74182743 -13.59598276 -27.21880082 -1.45187634 -9.94026059 0.08522134 8.72461793 2.27428968 -14.57863457 -5.50989162 15.78220719 34.24259822 34.04645561 2.49233660 10.90410676 -1.71713224 -5.02074849 7.79616245 -21.13256204 3.59017121 30.93056387 -6.63537748 -3.37099547 -29.77841631 -26.95790150 12.73810185 -17.89913485 25.11781341 5.17219635 -21.45705181 4.83248496 -4.77742021 -9.06356931 4.83250262 11.27752865 0.60240259 7.42457367 26.73366084 57.15084644 -1.89380284 -2.36398794 -11.10982911 2.87791991 -5.64597580 39.82306060 3.40297381 -9.49454404 -18.33080832 -16.12056429 -45.15449341 -5.18088325 -4.44905201 -14.57947391 -4.53438138 57.89615486 -6.41713106 18.49454294 -7.23303249 -7.77079744 -1.92920080 0.28333068 5.07906049 -7.26299520 -14.72053625 -0.62897481 6.34653380 -7.90109243 9.86666295 -22.31824816 9.78887354 9.36868484 -17.42920471 68.81587445 -6.75560045 33.78553977 13.74549848 -33.14997513 22.64904395 -18.1204042

NameError: name 'y_test_error' is not defined

In [7]:
print("The array: ",np.array(error_model))

The array:  [  3.01092292   3.3639307   -1.52740801 ... -14.56390467  10.399014
 -10.14220842]


error: 3.01092292 3.36393070 -1.52740801 -16.32560723 14.26725370 -15.80936774 52.68532597 -3.58451991 -1.41754858 38.89174808 -3.96511205 80.06407463 3.35607495 -24.04980557 -2.46362020 6.07587852 13.53466631 -3.79228732 -19.94479200 17.24180458 -4.67654290 -8.54180644 -18.72500673 -1.46999245 13.00421528 -5.75449794 7.65051168 -11.72277631 4.95467040 -2.99148438 -3.09865196 -17.60823864 5.16747943 -29.72758585 -22.79985823 -21.66836233 9.24823298 0.88228159 -1.19056880 -86.39863650 2.39579995 30.98138284 -9.06571755 -1.09645919 -0.09121263 -9.59703682 -63.79350441 15.46970658 22.50425298 -6.31541242 -11.30401029 -5.85006016 10.54138328 16.21710808 6.35232586 11.42580357 80.55695872 -2.18003599 -7.59156780 -15.26990970 8.19570886 -17.94041351 -29.04609092 11.37051210 12.61521075 -15.87995075 2.73284634 -13.35637183 1.97993594 20.63687937 -5.13130385 -8.15647756 -6.68268185 5.32751257 46.62727379 -3.13455032 -3.00273676 -66.09741638 6.14096581 -13.35923885 -21.82887865 -5.30799459 12.0

NameError: name 'y_test_error' is not defined